In [1]:
# Reload modules automatically on changes
%load_ext autoreload
%autoreload 2

#### Required import

In [2]:
import tsdf
import os
import sys
import numpy as np

#### Load dummy data and see its format

In [3]:
DATA_DIR = "../tests/data"
metadata = tsdf.load_metadata_from_path(os.path.join(DATA_DIR, "dummy_10_3_int16.json"))
data_in_1 = metadata["dummy_10_3_int16.bin"].load_binary()
print(f"Data type used for storing:\t {data_in_1.dtype}")
print(f"Data dimensions:\t\t {data_in_1.shape}")
print(f"Number of rows:\t\t\t {data_in_1.shape[0]}")
#print(data_in_1)

Data type used for storing:	 int16
Data dimensions:		 (10, 3)
Number of rows:			 10


#### Perform some data processing and visualise the new structure

In [4]:
data_out_1 = (data_in_1 / 10).astype('float32')
print(data_out_1.dtype, data_out_1.shape)
print(f"Data type used for storing:\t {data_out_1.dtype}")
print(f"Data dimensions:\t\t {data_out_1.shape}")
print(f"Number of rows:\t\t\t {data_out_1.shape[0]}")

float32 (10, 3)
Data type used for storing:	 float32
Data dimensions:		 (10, 3)
Number of rows:			 10


#### Visualise metadata available directly from the data (NumPy array)

In [5]:
bin_meta = tsdf.get_metadata_from_ndarray(data_out_1)
print(f"Binary formatting that can be inferred from the NumPy array:\n{bin_meta}")

Binary formatting that can be inferred from the NumPy array:
{'data_type': 'float', 'bits': 32, 'endianness': 'little', 'rows': 10}


#### Write the processed file data

#### Genarate binary
Generate binary data and save it and the corresponding TSDF metadata file.

In [6]:
path = os.path.join(DATA_DIR, "test_output_1.bin")
rs = np.random.RandomState(seed=42)
data = rs.rand(17, 1).astype(np.float32)

meta_file = "dummy_10_3_int16.json"
bin_file = "dummy_10_3_int16.bin"
path = os.path.join(DATA_DIR, meta_file)
metadata:tsdf.TSDFMetadata = tsdf.load_metadata_from_path(path)[bin_file]
#print(metadata)
print(metadata.__dict__)
local_path = os.path.join(DATA_DIR, "local_data")
new_meta =  tsdf.save_binary_file(local_path, bin_file, data, metadata.__dict__)
print(new_meta.__dict__)



{'_source_path': '/Users/vedran/git/biomarkersParkinon/tsdf/tests/data/dummy_10_3_int16.json', 'subject_id': 'dummy', 'study_id': 'dummy', 'device_id': 'dummy', 'endianness': 'little', 'metadata_version': '0.1', 'start_datetime_unix_ms': 1571135957025, 'start_iso8601': '2019-10-15T10:39:17.025000+00:00', 'end_datetime_unix_ms': 1571168851826, 'end_iso8601': '2019-10-15T19:47:31.826000+00:00', 'file_name': 'dummy_10_3_int16.bin', 'channels': ['x', 'y', 'z'], 'units': ['m/s/s', 'm/s/s', 'm/s/s'], 'scale_factors': [0.00469378, 0.00469378, 0.00469378], 'data_type': 'int', 'bits': 16, 'rows': 10}
{'_source_path': '/Users/vedran/git/biomarkersParkinon/tsdf/tests/data/dummy_10_3_int16.json', 'subject_id': 'dummy', 'study_id': 'dummy', 'device_id': 'dummy', 'endianness': 'little', 'metadata_version': '0.1', 'start_datetime_unix_ms': 1571135957025, 'start_iso8601': '2019-10-15T10:39:17.025000+00:00', 'end_datetime_unix_ms': 1571168851826, 'end_iso8601': '2019-10-15T19:47:31.826000+00:00', 'file

In [7]:
tsdf.save_metadata([new_meta])

!
{'_source_path': '/Users/vedran/git/biomarkersParkinon/tsdf/tests/data/dummy_10_3_int16.json', 'subject_id': 'dummy', 'study_id': 'dummy', 'device_id': 'dummy', 'endianness': 'little', 'metadata_version': '0.1', 'start_datetime_unix_ms': 1571135957025, 'start_iso8601': '2019-10-15T10:39:17.025000+00:00', 'end_datetime_unix_ms': 1571168851826, 'end_iso8601': '2019-10-15T19:47:31.826000+00:00', 'file_name': 'dummy_10_3_int16.bin', 'channels': ['x', 'y', 'z'], 'units': ['m/s/s', 'm/s/s', 'm/s/s'], 'scale_factors': [0.00469378, 0.00469378, 0.00469378], 'data_type': 'float', 'bits': 32, 'rows': 17}


In [63]:
meta = extract_fields(original_metadata, ['study_id', 'sefsef'])

meta_1 = tsdf.save_binary_file('dummy_output.bin', data_out_1)



new_metadata = MetaData([meta, bin_meta])

new_metadata = MetaData({
    study_id: original_metadata['study_id'],
    ...
    bits: bin_meta['bits']
    ...
})

SyntaxError: invalid syntax (1896731653.py, line 12)